In [2]:
import mnist
import scipy.misc
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import numpy as np
import datetime
np.set_printoptions(suppress=True)
from os import listdir
from os.path import isfile, join
from math import*
from decimal import Decimal
from collections import defaultdict
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
#scipy.misc.toimage(scipy.misc.imresize(images[0,:,:] * -1 + 256, 10.))

train_images = mnist.train_images()
train_labels = mnist.train_labels()

test_images = mnist.test_images()
test_labels = mnist.test_labels()

## Shift-and-scale normalization:

In [3]:
scaler              = StandardScaler()

scaled_train_images = scaler.fit_transform(train_images.reshape\
                                     ((train_images.shape[0], train_images.shape[1] * train_images.shape[2])))
scaled_test_images  = scaler.fit_transform(test_images.reshape\
                                     ((test_images.shape[0], test_images.shape[1] * test_images.shape[2])))

print ('Max: ' + str(scaled_test_images.max()))
print ('Min: ' + str(scaled_test_images.min()))
print ('Mean: %f' % (scaled_test_images.mean()))
print ('Variance: %f' % (scaled_test_images.var()))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Max: 99.994999875
Min: -1.31006468485
Mean: 0.000000
Variance: 0.852041


## Zero mean, unit variance:

In [4]:
print ('Original Max: ' + str(train_images.max()))
print ('Original Min: ' + str(train_images.min()))

scaler  = MinMaxScaler(feature_range=(0, 1))
scaled_train_images = scaler.fit_transform(train_images.reshape\
                                          ((train_images.shape[0], train_images.shape[1] * train_images.shape[2])))

print ('Max: ' + str(scaled_train_images.max()))
print ('Min: ' + str(scaled_train_images.min()))
print ('Mean: %f' % (scaled_train_images.mean()))
print ('Variance: %f' % (scaled_train_images.var()))

Original Max: 255
Original Min: 0


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Max: 1.0
Min: 0.0
Mean: 0.130663
Variance: 0.094932


##  20 NG KNN

In [3]:
news_train = fetch_20newsgroups(data_home='../data/20newsgroups/', 
                                subset='train', 
                                remove=('headers', 'footers', 'quotes'))
news_test = fetch_20newsgroups(data_home='../data/20newsgroups/', 
                                subset='test', 
                                remove=('headers', 'footers', 'quotes'))

In [7]:
vectorizer                = CountVectorizer(binary=True, stop_words='english')
counts_train              = vectorizer.fit_transform(news_train.data)
counts_test               = vectorizer.transform(news_test.data)
cos_similarity_train      = cosine_similarity(counts_train)
cos_similarity_train_test = cosine_similarity(X=counts_test, Y=counts_train)

In [21]:
vectorizer.get_feature_names()

['00',
 '000',
 '0000',
 '00000',
 '000000',
 '00000000',
 '0000000004',
 '00000000b',
 '00000001',
 '00000001b',
 '00000010',
 '00000010b',
 '00000011',
 '00000011b',
 '00000074',
 '00000093',
 '000000e5',
 '00000100',
 '00000100b',
 '00000101',
 '00000101b',
 '00000110',
 '00000110b',
 '00000111',
 '00000111b',
 '000005102000',
 '00000510200001',
 '00000ee5',
 '00001000',
 '00001000b',
 '00001001',
 '00001001b',
 '00001010',
 '00001010b',
 '00001011',
 '00001011b',
 '000010af',
 '00001100',
 '00001100b',
 '00001101',
 '00001101b',
 '00001110',
 '00001110b',
 '00001111',
 '00001111b',
 '000042',
 '000062david42',
 '000094',
 '0001',
 '00010000',
 '00010000b',
 '00010001',
 '00010001b',
 '00010010',
 '00010010b',
 '00010011',
 '00010011b',
 '000100255pixel',
 '00010100',
 '00010100b',
 '00010101',
 '00010101b',
 '00010110',
 '00010110b',
 '00010111',
 '00010111b',
 '00011000',
 '00011000b',
 '00011001',
 '00011001b',
 '00011010',
 '00011010b',
 '00011011',
 '00011011b',
 '00011100',
 '

In [18]:
def knn(index, k, similarity_matrix):
    return news_train.target[similarity_matrix[index].argsort()[-(k + 1):-1]]

In [25]:
list(knn(990, 5, cos_similarity_train))

[19, 4, 3, 4, 18]

In [33]:
news_train.target_names[19]

'talk.religion.misc'

In [35]:
news_train.target_names[4]

'comp.sys.mac.hardware'

In [36]:
news_train.target_names[3]

'comp.sys.ibm.pc.hardware'

In [48]:
news_train_len = len(news_train.data)
news_test_len  = len(news_test.data)
k              = 5

def knn(index, k, similarity_matrix):
    print (news_train.target[np.argmax(similarity_matrix[index].argsort()[-(k + 1):-1])])
    print ('\n\n\n')
    return news_train.target[np.argmax(similarity_matrix[index].argsort()[-(k + 1):-1])]

train_predict = []

for doc_index in range(news_train_len):
    y = news_train.target[doc_index]
    yhat = knn(doc_index, k, cos_similarity_train)
    train_predict.append(y == yhat)
    
print ('The training accuracy is: ' + str(np.mean(train_predict)))
    
test_predict = []

for doc_index in range(news_test_len):
    y = news_test.target[doc_index]
    yhat = knn(doc_index, k, cos_similarity_train_test)
    test_predict.append(y == yhat)
    
print ('The testing accuracy is: ' + str(np.mean(test_predict)))



4




7




1




4




4




14




7




7




4




4




4




7




1




7




4




1




14




4




4




4




1




1




1




14




4




4




4




14




14




7




7




7




4




4




7




1




4




4




1




7




14




14




4




4




7




4




1




14




4




4




14




14




14




7




7




4




1




4




7




7




1




1




14




7




4




4




14




4




7




1




1




4




4




1




4




1




4




4




4




7




4




4




1




14




1




4




1




4




14




14




4




4




4




4




14




4




7




4




14




4




4




4




4




7




4




1




4




1




1




1




4




14




4




14




14




4




4




4




4




1




1




4




1




7




4




1




1




7




4




14




14




7




14




4




4




1




4




4




4




4




14




14




7




1




7




7




1




1




1




4




7




14




7




7




1




1




1




4




1




4




4




4




7

4




1




14




1




4




4




4




1




1




4




14




14




14




7




1




14




4




4




4




7




4




7




4




14




7




4




14




14




7




7




14




4




4




14




4




4




1




14




4




7




14




4




4




7




1




1




1




4




7




4




4




1




4




4




1




4




4




7




7




4




7




4




1




7




14




14




14




1




14




4




14




4




4




4




4




7




1




4




1




14




1




7




7




4




1




1




1




1




14




14




4




4




14




4




14




1




14




4




1




1




14




7




4




4




1




14




7




7




14




4




7




7




7




4




7




1




1




14




4




4




14




7




7




7




1




4




4




4




4




14




7




7




14




1




1




14




14




4




7




4




1




1




14




14




14




4




1




7




7




4




14




14




7




4




1




4




14




4




1




7




14

7




1




1




7




14




4




4




14




4




14




7




7




14




1




14




4




7




4




7




4




1




4




4




14




7




14




14




7




4




1




7




14




14




1




14




1




14




4




4




14




1




1




4




14




4




14




1




4




1




4




14




4




1




1




1




1




4




7




7




1




14




4




7




4




1




7




4




4




1




4




1




1




14




1




4




7




4




14




4




4




4




4




4




4




4




4




7




1




4




14




4




7




1




4




7




1




7




4




7




14




4




14




4




1




14




14




4




14




4




1




14




14




4




1




4




7




4




7




7




14




4




4




1




4




7




4




14




7




1




14




4




4




4




14




1




1




4




14




7




7




14




1




14




4




14




14




1




7




7




4




4




4




4




7




4




14




4




4




1




4




7



1




1




14




4




4




4




14




1




14




4




1




4




1




4




14




1




14




7




14




1




4




4




4




4




4




14




4




4




4




4




4




14




4




4




4




7




4




7




14




4




1




4




7




4




14




7




4




1




1




4




14




4




4




4




1




1




1




1




14




14




7




4




1




7




7




14




4




4




4




4




14




7




7




7




7




14




14




14




14




4




7




7




4




4




1




1




1




4




4




1




14




1




4




4




4




7




4




14




14




4




1




4




7




14




7




4




1




4




4




14




1




1




14




1




4




1




14




4




4




7




7




4




1




14




7




7




4




4




7




4




7




7




4




4




14




14




14




7




4




14




1




1




14




4




4




7




1




4




14




4




4




4




1




4




14




7




4




7




4




4




7






7




7




4




4




1




4




1




1




1




14




1




7




14




7




4




4




14




1




7




4




4




4




14




4




4




4




14




4




4




4




7




4




14




1




7




7




14




7




14




4




1




4




4




1




7




7




4




1




7




4




14




4




14




7




4




1




14




4




1




4




14




7




4




1




14




1




1




14




1




7




14




4




14




14




1




7




1




4




1




4




1




4




4




4




4




1




4




4




4




4




14




14




7




4




7




4




14




4




1




1




4




1




7




4




1




4




1




4




4




4




1




7




14




14




4




7




14




7




7




14




14




4




4




1




4




4




7




1




14




14




14




1




4




4




4




4




1




4




14




4




4




4




7




4




14




14




4




4




1




14




1




1




7




4




1




1




14




14




4




14




4





...

14




4




4




14




4




14




4




14




4




1




1




4




4




1




14




7




7




4




4




7




4




7




4




4




14




1




4




4




1




4




4




14




1




7




7




14




4




4




7




7




1




1




1




4




7




4




14




4




7




1




4




7




4




14




4




14




4




1




14




1




7




14




4




14




4




14




1




14




7




1




7




7




4




4




1




14




4




4




7




1




14




14




4




4




14




4




7




4




14




14




1




4




1




4




4




4




1




4




7




14




14




1




14




14




4




14




7




1




14




4




14




7




14




4




7




4




1




4




14




7




4




4




1




1




14




4




7




7




14




14




7




1




7




14




4




14




4




7




4




1




14




1




4




14




14




14




14




4




1




14




1




14




4




14




4




7




14




14




7






1




4




4




4




7




4




4




4




14




1




1




1




7




1




7




4




4




4




1




7




7




7




14




4




14




4




14




14




4




14




7




4




1




14




4




14




14




14




14




1




4




14




7




1




4




14




4




1




1




4




7




1




1




7




1




7




4




4




1




4




7




4




4




4




7




1




14




4




4




7




4




4




4




4




14




14




4




14




4




7




14




1




1




1




1




14




1




14




4




4




7




4




14




14




1




4




14




4




1




7




7




14




4




1




4




7




1




7




7




4




4




7




7




7




14




1




1




4




1




7




14




4




14




4




4




4




7




7




4




7




4




7




7




14




7




4




14




7




1




14




1




1




4




1




4




4




14




4




14




4




4




7




4




4




14




14




7




14




4




14




4




14




4




7




7




1




14




7




7




14




4




4




14




14




7




4




4




14




4




1




7




4




7




14




1




4




7




4




4




4




4




7




14




4




4




4




4




14




1




4




14




4




4




4




4




14




4




14




1




4




4




14




4




14




14




4




4




1




4




14




4




7




7




1




1




4




7




14




14




7




4




1




4




1




1




4




4




14




4




4




14




7




7




1




4




4




4




4




7




14




7




7




4




7




7




4




1




4




1




14




1




1




4




14




7




4




14




1




14




1




1




14




7




7




1




7




4




14




7




7




4




1




4




4




7




4




4




14




4




4




7




14




1




1




4




4




7




4




7




4




1




14




4




4




4




7




1




14




4




14




7




1




7




7




1




4




7




7




14




4




14




7






4




4




4




4




4




14




1




7




1




4




4




4




1




4




14




4




14




7




4




4




4




4




4




4




14




4




4




1




1




7




14




4




4




4




4




14




4




14




7




14




14




1




14




7




14




1




7




4




4




1




1




7




4




14




1




4




1




1




14




1




4




1




4




1




1




1




1




14




1




1




4




4




4




7




4




14




4




7




7




14




1




4




4




1




4




14




4




4




14




4




14




1




7




1




14




4




7




14




4




14




4




7




7




14




14




7




14




14




4




1




1




7




1




1




4




4




4




4




4




7




4




4




14




7




4




14




4




7




4




4




7




14




14




1




1




1




7




7




7




14




4




14




14




1




4




1




14




14




4




4




4




4




4




4




14




1




1




7




14




4




4




1




7




4




1




7




1




4




7




4




14




1




7




4




4




14




4




1




1




14




1




1




4




4




1




14




14




4




4




4




4




14




1




14




7




7




1




1




14




4




1




7




4




1




7




1




14




7




1




14




4




4




4




1




4




1




1




14




4




4




14




1




1




4




14




4




14




4




1




1




14




1




1




4




14




4




7




1




4




4




1




4




4




14




4




4




4




4




4




1




14




7




4




7




1




4




4




1




7




7




4




7




4




4




7




7




4




1




4




1




1




7




7




7




4




4




7




14




4




14




4




4




1




7




1




7




7




14




7




4




14




7




14




7




1




4




1




1




4




4




4




7




4




1




7




7




14




1




4




14




14




14




1




4




7




4




1




4




14




4




4




4




7





In [45]:
similarity_matrix[index].argsort()[-(k + 1):-1])

NameError: name 'similarity_matrix' is not defined

In [24]:
choice_maker               = np.random.choice([True, False], len(scaled_train_images), p = [0.4, 0.6])
scaled_train_images_sample = scaled_train_images[choice_maker]
train_label_sample         = train_labels[choice_maker]

print(scaled_train_images_sample.shape)
print(train_label_sample.shape)

print(scaled_test_images.shape)
print(test_labels.shape)

(24022, 784)
(24022,)
(10000, 784)
(10000,)


In [28]:
euc_similarity_matrix_train  = euclidean_distances(X=scaled_train_images_sample, Y=scaled_train_images_sample)
cos_similarity_matrix_train  = cosine_similarity(X=scaled_train_images_sample, Y=scaled_train_images_sample)

In [30]:
euc_similarity_matrix_test  = euclidean_distances(X=scaled_test_images, Y=scaled_train_images_sample)
cos_similarity_matrix_test  = cosine_similarity(X=scaled_test_images, Y=scaled_train_images_sample)

In [20]:
#np.save('../data/similarity_matrix_coz', cos_similarity_matrix)
#cos_similarity_matrix = np.load('../data/similarity_matrix_coz.npy')



In [8]:
scaled_train_images_sample_array = np.array(scaled_train_images_sample)
scaled_test_images_array = np.array(scaled_test_images)
train_label_sample_array = np.array(train_label_sample)
test_labels_array = np.array(test_labels)

In [9]:
def mnist_knn(index, k, similarity_matrix):
    return np.array(similarity_matrix[index,:]).argsort()[-(k+1):-1]

## Cosine

In [62]:
predict = []
for index in range(len(scaled_train_images_sample)):
    
    #print (index)    
    y_select = train_label_sample_array[index]
    yhat = max(train_label_sample_array[mnist_knn(index, 5, cos_similarity_matrix_train)])
    predict.append(y_select == yhat)

In [65]:
np.mean(predict)

0.92794860027535564

In [16]:
test_predict = []
for index in range(len(scaled_test_images_array)):
    #print (mnist_knn(index, 5, cos_similarity_matrix_test))
    y_select = test_labels[index]
    yhat = max(train_label_sample_array[mnist_knn(index, 5, cos_similarity_matrix_test)])
    test_predict.append(y_select == yhat)

In [17]:
np.mean(test_predict)

0.91279999999999994

## Euclidean

In [35]:
test_predict = []
for index in range(len(scaled_test_images_array)):
    #print (mnist_knn(index, 5, cos_similarity_matrix_test))
    y_select = test_labels[index]
    yhat = min(train_label_sample_array[mnist_knn(index, 5, euc_similarity_matrix_test)])
    test_predict.append(y_select == yhat)

In [36]:
np.mean(test_predict)

0.84730000000000005